In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pathlib import Path

In [ ]:
input_path= Path("/kaggle/input/tabular-playground-series-jun-2022/")

In [ ]:
data= pd.read_csv(input_path / 'data.csv', index_col='row_id')
print("input shape:",data.shape)
print(" input nan count:",data.isna().sum().sum())

In [ ]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(
missing_values=np.nan, strategy="mean")
output = data.copy()
output[:]=imp.fit_transform(data)
print('output shape:',output.shape)
print('output nan count:',output.isna().sum().sum())

In [ ]:
%%time

def generate_submission(source_df: pd.DataFrame, output_df: pd.DataFrame) -> pd.DataFrame:
    # Melt source dataframe filtered on NaN values to form [row_id, col, isNull] ...
    # ... with MultiIndex on (row_id, col)
    nan_only = (source_df
                .isna()
                .melt(ignore_index=False, var_name='col', value_name='isNull')
                .query('isNull == True')
                .set_index(['col'], append=True))
       # Melt output dataframe to form [row_id, col, value] with MultiIndex on (row_id, col)
    out = (output_df
            .melt(ignore_index=False, var_name='col')
            .set_index(['col'], append=True))
     # Filter output's MultiIndex on nan_only's MultiIndex
    out = (out.loc[nan_only.index]
               .sort_index())
        # Flatten MultiIndex to Index & rename to desired column
    out.index = [f'{r}-{c}' for r, c in out.index]
    out.index.name = 'row-col'
    return out

In [ ]:
%%time

def generate_submission(source_df: pd.DataFrame, output_df: pd.DataFrame) -> pd.DataFrame:
    # Melt source dataframe filtered on NaN values to form [row_id, col, isNull] ...
    # ... with MultiIndex on (row_id, col)
    nan_only = (source_df
                .isna()
                .melt(ignore_index=False, var_name='col', value_name='isNull')
                .query('isNull == True')
                .set_index(['col'], append=True))

    # Melt output dataframe to form [row_id, col, value] with MultiIndex on (row_id, col)
    out = (output_df
            .melt(ignore_index=False, var_name='col')
            .set_index(['col'], append=True))

    # Filter output's MultiIndex on nan_only's MultiIndex
    out = (out.loc[nan_only.index]
               .sort_index())
    
    # Flatten MultiIndex to Index & rename to desired column
    out.index = [f'{r}-{c}' for r, c in out.index]
    out.index.name = 'row-col'
    return out

In [ ]:
result = generate_submission(data, output)
result

In [ ]:
submission = pd.read_csv(input_path / 'sample_submission.csv',index_col='row-col')

In [ ]:
submission